In [19]:
import pandas as pd

from sys import platform

if platform == "win32":
    path = 'C:/Users/olive/GitHub/f1-analytics/'
elif platform == "darwin":
    path = '~/Documents/GitHub/f1-analytics/'
    # path = '/Users/oliverjcarter/Documents/GitHub/f1-analytics/'

In [20]:
merged = pd.read_csv(path+'data/merged.csv')

In [21]:
df = merged.drop(['country', 'lat', 'long', 'date', 'date_of_birth', 'nationality'], axis=1)

In [22]:
df.rename(columns={'driver_points': 'driver_points_after', 
    'constructor_points': 'constructor_points_after', 
    'constructor_wins': 'constructor_wins_after', 
    'driver_wins': 'driver_wins_after', 
    'constructor_standings_pos': 'constructor_standings_pos_after',
    'driver_standings_pos': 'driver_standings_pos_after'}, inplace=True)

In [32]:
points_before = {
    'season': [],
    'round': [],
    'driver': [],
    'driver_points_before': [],
    'constructor_points_before': []
}

In [33]:
grouped = df.groupby(['season'])

for season, group in grouped:
    drivers = group.driver.unique()
    for driver in drivers:
        q = group.query('driver ==@driver')
        for index in list(range(len(q))):
            points_before['season'].append(season)
            points_before['round'].append(index+1)
            points_before['driver'].append(driver)
            if index != 0:
                points_before['driver_points_before'].append(q.iloc[index-1].driver_points_after)
                points_before['constructor_points_before'].append(q.iloc[index-1].constructor_points_after)
            else:
                points_before['driver_points_before'].append(0.0)
                points_before['constructor_points_before'].append(0.0)


points_before = pd.DataFrame(points_before)
points_before

/var/folders/77/g3_0sy3x3y5_z5nv9dmdby380000gn/T/ipykernel_34463/1233068639.py:3: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for season, group in grouped:


,season,round,driver,driver_points_before,constructor_points_before
0,2014,1,nico_rosberg,0.0,0.0
1,2014,2,nico_rosberg,25.0,25.0
2,2014,3,nico_rosberg,43.0,68.0
3,2014,4,nico_rosberg,61.0,111.0
4,2014,5,nico_rosberg,79.0,154.0
...,...,...,...,...,...
3614,2022,16,sebastian_vettel,24.0,37.0
3615,2022,17,sebastian_vettel,32.0,45.0
3616,2022,18,sebastian_vettel,36.0,49.0
3617,2022,19,sebastian_vettel,36.0,49.0


In [41]:
test = merged.query('season == 2022 & driver == "sebastian_vettel"')
test[['season', 'round', 'podium','driver_points']]

,season,round,podium,driver_points
3243,2022,3,19,0
3252,2022,4,8,4
3278,2022,5,17,4
3292,2022,6,11,4
3311,2022,7,10,5
3327,2022,8,6,13
3353,2022,9,12,13
3370,2022,10,9,15
3397,2022,11,17,15
3411,2022,12,11,15


In [42]:
df1 = pd.merge(df, points_before, how='inner', on=['season', 'round', 'driver'])
df1[['season', 'round','driver', 'driver_points_before','driver_points_after']].query('season == 2022 & driver == "sebastian_vettel"')

,season,round,driver,driver_points_before,driver_points_after
3126,2022,3,sebastian_vettel,4.0,0
3135,2022,4,sebastian_vettel,4.0,4
3161,2022,5,sebastian_vettel,4.0,4
3175,2022,6,sebastian_vettel,5.0,4
3194,2022,7,sebastian_vettel,13.0,5
3210,2022,8,sebastian_vettel,13.0,13
3236,2022,9,sebastian_vettel,15.0,13
3253,2022,10,sebastian_vettel,15.0,15
3280,2022,11,sebastian_vettel,15.0,15
3294,2022,12,sebastian_vettel,16.0,15


In [29]:
points_from = {
    'season': [],
    'round': [],
    'driver': [],
    'driver_points_from': [],
    'constructor_points_from': []
}

In [30]:
grouped = df1.groupby(['season'])

for season, group in grouped:
    drivers = group.driver.unique()
    for driver in drivers:
        q = group.query('driver ==@driver')
        for index in list(range(len(q))):
            n = q.iloc[index]
            points_from['season'].append(season)
            points_from['round'].append(index+1)
            points_from['driver'].append(driver)
            points_from['driver_points_from'].append(n.driver_points_after - n.driver_points_before)
            points_from['constructor_points_from'].append(n.constructor_points_after - n.constructor_points_before)
            

points_from = pd.DataFrame(points_from)

/var/folders/77/g3_0sy3x3y5_z5nv9dmdby380000gn/T/ipykernel_34463/748613896.py:3: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for season, group in grouped:


In [31]:
points_from

,season,round,driver,driver_points_from,constructor_points_from
0,2014,1,nico_rosberg,25.0,25.0
1,2014,2,nico_rosberg,18.0,43.0
2,2014,3,nico_rosberg,18.0,43.0
3,2014,4,nico_rosberg,18.0,43.0
4,2014,5,nico_rosberg,18.0,43.0
...,...,...,...,...,...
3486,2022,14,sebastian_vettel,-4.0,-12.0
3487,2022,15,sebastian_vettel,-8.0,-8.0
3488,2022,16,sebastian_vettel,-4.0,-4.0
3489,2022,17,sebastian_vettel,0.0,0.0


In [37]:
driver_points_per_race = []
driver_points_from_race = []

constructor_points_per_race = []
constructor_points_from_race = []

for index, row in df.iterrows():
    driver_points_per_race.append(round(row.driver_points_before_race / row['round'], 2))
    driver_points_from_race.append(round(row.driver_points_after_race - row.driver_points_before_race, 1))

    constructor_points_per_race.append(round(row.constructor_points_before_race / row['round'], 2))
    constructor_points_from_race.append(round(row.constructor_points_after_race - row.constructor_points_before_race, 1))

In [38]:
df['driver_points_per_race'] = driver_points_per_race
df['driver_points_from_race'] = driver_points_from_race

df['constructor_points_per_race'] = constructor_points_per_race
df['constructor_points_from_race'] = constructor_points_from_race

In [39]:
test = df[['season', 'round', 'driver', 'driver_points_before_race', 'driver_points_after_race', 'driver_points_from_race']]

In [40]:
test.query('season == 2021 & driver == "lewis_hamilton"')

,season,round,driver,driver_points_before_race,driver_points_after_race,driver_points_from_race
2760,2021,1,lewis_hamilton,0.0,25,25.0
2781,2021,2,lewis_hamilton,25.0,44,19.0
2799,2021,3,lewis_hamilton,44.0,69,25.0
2819,2021,4,lewis_hamilton,69.0,94,25.0
2845,2021,5,lewis_hamilton,94.0,101,7.0
2871,2021,6,lewis_hamilton,101.0,101,0.0
2877,2021,7,lewis_hamilton,101.0,119,18.0
2895,2021,8,lewis_hamilton,119.0,138,19.0
2917,2021,9,lewis_hamilton,138.0,150,12.0
2934,2021,10,lewis_hamilton,150.0,177,27.0


In [92]:
grouped = df.groupby(['season', 'round'])
last_3_races = []

for index, group in grouped:
        for _, row in group.iterrows():
                r= index[1]
                rounds = list(range(r-1, r-4, -1))
                points = df.query(
                        'season ==@index[0] & round in @rounds & driver ==@row.driver'
                        ).agg({
                                'driver_points_from_race': sum
                                }).values[0]

                last_3_races.append(points)

[0, -1, -2]
[0, -1, -2]
[0, -1, -2]
[0, -1, -2]
[0, -1, -2]
[0, -1, -2]
[0, -1, -2]
[0, -1, -2]
[0, -1, -2]
[0, -1, -2]
[0, -1, -2]
[0, -1, -2]
[0, -1, -2]
[0, -1, -2]
[0, -1, -2]
[0, -1, -2]
[0, -1, -2]
[0, -1, -2]
[0, -1, -2]
[0, -1, -2]
[1, 0, -1]
[1, 0, -1]
[1, 0, -1]
[1, 0, -1]
[1, 0, -1]
[1, 0, -1]
[1, 0, -1]
[1, 0, -1]
[1, 0, -1]
[1, 0, -1]
[1, 0, -1]
[1, 0, -1]
[1, 0, -1]
[1, 0, -1]
[1, 0, -1]
[1, 0, -1]
[1, 0, -1]
[1, 0, -1]
[1, 0, -1]
[1, 0, -1]
[1, 0, -1]
[1, 0, -1]
[2, 1, 0]
[2, 1, 0]
[2, 1, 0]
[2, 1, 0]
[2, 1, 0]
[2, 1, 0]
[2, 1, 0]
[2, 1, 0]
[2, 1, 0]
[2, 1, 0]
[2, 1, 0]
[2, 1, 0]
[2, 1, 0]
[2, 1, 0]
[2, 1, 0]
[2, 1, 0]
[2, 1, 0]
[2, 1, 0]
[2, 1, 0]
[2, 1, 0]
[2, 1, 0]
[2, 1, 0]
[3, 2, 1]
[3, 2, 1]
[3, 2, 1]
[3, 2, 1]
[3, 2, 1]
[3, 2, 1]
[3, 2, 1]
[3, 2, 1]
[3, 2, 1]
[3, 2, 1]
[3, 2, 1]
[3, 2, 1]
[3, 2, 1]
[3, 2, 1]
[3, 2, 1]
[3, 2, 1]
[3, 2, 1]
[3, 2, 1]
[3, 2, 1]
[3, 2, 1]
[3, 2, 1]
[4, 3, 2]
[4, 3, 2]
[4, 3, 2]
[4, 3, 2]
[4, 3, 2]
[4, 3, 2]
[4, 3, 2]
[4, 3, 2]
[4, 3, 2

In [42]:
grouped = df.groupby(['season', 'round'])
points_before_race = []

for index, group in grouped:
        for _, row in group.iterrows():
                r = index[1]
                query = df.query(
                        'season ==@index[0] & round ==@r-1 & driver ==@row.driver')

                points = query.driver_points_after_race
                if len(points) != 0:
                        points_before_race.append(points.values[0])
                else:
                        points_before_race.append(0.0)

In [43]:
df['driver_points_last_3_races'] = last_3_races
df['driver_points_before_race'] = points_before_race

In [44]:
### Removing sprint races

# df = df[~
# (
#     ((df.season == 2014) & (df['round'] == 19))
#     |
#     ((df.season == 2021) & (df['round'].isin([10, 14, 19])))
#     |
#     ((df.season == 2022) & (df['round'].isin([4, 11, 21])))
# )
# ]

In [45]:
testing = df[['season','round', 'driver', 'driver_points_before_race', 'driver_points_from_race', 'driver_points_last_3_races']]

In [46]:
testing.query('season == 2021 & driver == "max_verstappen"')

,season,round,driver,driver_points_before_race,driver_points_from_race,driver_points_last_3_races
2761,2021,1,max_verstappen,0.0,18.0,0.0
2780,2021,2,max_verstappen,18.0,25.0,18.0
2800,2021,3,max_verstappen,43.0,18.0,43.0
2820,2021,4,max_verstappen,61.0,19.0,61.0
2839,2021,5,max_verstappen,80.0,25.0,62.0
2874,2021,6,max_verstappen,105.0,0.0,62.0
2876,2021,7,max_verstappen,105.0,26.0,44.0
2894,2021,8,max_verstappen,131.0,25.0,51.0
2914,2021,9,max_verstappen,156.0,26.0,51.0
2953,2021,10,max_verstappen,182.0,3.0,77.0


In [47]:
df[['driver', 'constructor', 
        # 'podium',
    #    'driver_points_after_race', 'driver_wins_after_race',
    #    'driver_standings_pos_after_race', 'constructor_points_after_race',
    #    'constructor_wins_after_race', 'constructor_standings_pos_after_race',
    #    'qualifying_pos',
        # 'starting_grid',
    #    'qual_time', 
       'stage', 'q_delta']].groupby('stage').describe()

q_delta                                                          
        count      mean       std  min      25%     50%     75%     max
stage                                                                  
q1      926.0  3.695961  3.112564  0.0  2.38150  3.0690  3.9105  33.846
q2      932.0  2.270960  1.635836  0.0  1.57675  2.1005  2.5380  26.141
q3     1761.0  1.235083  1.569444  0.0  0.32000  0.9150  1.5680  22.112

In [48]:
# df.to_csv(path+'data/processed.csv', index=False)